<a href="https://colab.research.google.com/github/SiYeon-Ju/Colab_work/blob/main/Cobrapy_OB3b_220610_fin_v10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !python --version ## 3.7 버전에서 구동 확인 ##
!pip install cobra==0.17.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.8 MB/s 
     |████████████████████████████████| 43.9 MB 3.5 MB/s 
     |████████████████████████████████| 109 kB 52.6 MB/s 
     |████████████████████████████████| 147 kB 24.3 MB/s 
     |████████████████████████████████| 2.3 MB 8.1 MB/s 
     |████████████████████████████████| 546 kB 49.1 MB/s 


In [2]:
!git clone https://github.com/ensakz/gem_methylosinus_trichosporium.git

Cloning into 'gem_methylosinus_trichosporium'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 68 (delta 13), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (68/68), done.


In [3]:
#### Cobrapy # 0.17.0 버전 Doc
### https://cobrapy.readthedocs.io/en/0.17.1/configuration.html
#### Cobrapy # latest 버전 Doc
### https://cobrapy.readthedocs.io/en/latest/index.html


### 라이브러리 불러오기 ###
import cobra
import pandas as pd
import numpy as np

### 함수 추가 ###

## 모델 정보 저장 ##
# id, reaction, name, lower_bound, upper_bound, gene_reaction_rule #
def model_info(model): 
  lst = []
  for i in range(len(model.reactions)):
    lst.append([model.reactions[i].id, model.reactions[i].reaction, model.reactions[i].name, 
    model.reactions[i].lower_bound, model.reactions[i].upper_bound,  model.reactions[i].gene_reaction_rule])
  global info 
  info = pd.DataFrame( lst, columns = ['ID', 'Rxn', 'Gene_name', 'LB','UB','Gene_Rxn_Rule'] )
  info.to_csv(str(model) + '_Model_info.csv', header = True)
  return print('Model_info save complete!')

## 모델 정보 + Flux 저장 ##
def model_flux(model, sol): 
  lst = []
  for i in range(len(model.reactions)):
    lst.append([model.reactions[i].id, model.reactions[i].reaction, model.reactions[i].name, 
    model.reactions[i].lower_bound, model.reactions[i].upper_bound,  model.reactions[i].gene_reaction_rule])
  global info 
  info = pd.DataFrame( lst, columns = ['ID', 'Rxn', 'Gene_name', 'LB','UB','Gene_Rxn_Rule'] )
  global flux
  flux_data = pd.DataFrame([sol.fluxes.index, sol.fluxes], index = ['ID', 'Flux']).transpose()
  Merging_data = pd.merge(info, flux_data, on ='ID')
  flux = Merging_data[['ID', 'Rxn', 'Flux','LB', 'UB', 'Gene_Rxn_Rule', 'Gene_name']]
  flux.to_csv(str(model)+'_'+ str(sol) +'_Flux_results.csv', header = True)
  return print('Model_flux save complete!')

In [4]:
## Json 모델 불러오기 ##
model_dir = r'/content/gem_methylosinus_trichosporium/model_files/iMsOB3b.json'
iMsOB3b = cobra.io.load_json_model(model_dir)

# Bound 설정 #
iMsOB3b.reactions.EX_ch4_e.bounds = (-14.9, 0)
iMsOB3b.reactions.EX_no3_e.bounds = (-1000, 0)

# FBA 축약본 보기 #
iMsOB3b.summary()
   
# 모델 정보 저장 ('info'로 flux확인 가능)   
model_info(iMsOB3b)

Model_info save complete!


In [ ]:
## 모델에 새 반응식 추가하는 방법..##
# iMsOB3b_v2 = cobra.io.load_json_model(model_dir)
# JSY = iMsOB3b_v2.reactions.PHAS
# JSY.id = 'JSY'
# JSY.reaction = "ch4_p + o2_p + qh2_c --> h2o_p + meoh_p + q_c"
# iMsOB3b.add_reaction(JSY)
# model_info(iMsOB3b)


## 아애 반응식 지우는법 ##
# iMsOB3b.reactions.JSY.remove_from_model()
# model_info(iMsOB3b)

Model_info save complete!


In [5]:
## 모델의 Bounds 설정 ##
iMsOB3b.reactions.EX_ch4_e.bounds = (-14.9, 0)  ## 메탄 uptake rates
iMsOB3b.reactions.FALDA.bounds = (0, 0)         ## THF 관련 유전자
iMsOB3b.reactions.EX_nh4_e.bounds = (0, 0)      ## 암모니아 바운드 0
iMsOB3b.reactions.EX_n2_e.bounds = (0, 0)       ## 질소 바운드 0
iMsOB3b.reactions.EX_no3_e.bounds = (-1000, 0)  ## 질산 바운드 -inf ## 과제팀은 이거씀

iMsOB3b.reactions.PHBD2.bounds = (0, 0) ## pbhb_c 분해 경로 제거

## objective function 변경 및 확인 ##
iMsOB3b.objective = 'PHAS'
print('print objective:', iMsOB3b.objective.expression)

## FBA 축약본 보기 ##
iMsOB3b.summary()

####### FBA analysis #########
FBA_sol = iMsOB3b.optimize()
#solution.fluxes

## pFBA flux값 저장 ###
model_flux(iMsOB3b, FBA_sol) #### 함수 model_flux에  Model과 Sol 넣으면 저장됨 ###

print objective: 1.0*PHAS - 1.0*PHAS_reverse_71821
Model_flux save complete!


In [ ]:
## Parsimonas FBA 분석 ##
pFBA_sol_ = cobra.flux_analysis.pfba(iMsOB3b,objective = 'PHAS')

## pFBA flux값 저장 ###
model_flux(iMsOB3b, pFBA_sol_)

## FBA와 pFBA간에 차이 확인 ##
print('FBA - pFBA',abs(FBA_sol.fluxes["PHAS"] - pFBA_sol_.fluxes["PHAS"]))

Model_flux save complete!
FBA - pFBA 1.1796119636642288e-16


In [ ]:
## Loopless FBA 분석 ##
#### 아래건 이상하게 안돌아간다...####
# from cobra.flux_analysis.loopless import add_loopless, loopless_solution
# print('start')
# with iMsOB3b:
#     add_loopless(iMsOB3b)
#     try:
#         print(iMsOB3b.solver)
#         solution = iMsOB3b.optimize()
#     except:
#         print('model is infeasible')
# print("loopless solution: status = " + solution.status)
# print("loopless solution flux: v3 = %.1f" % solution.fluxes["v3"])

## If there is no forced flux through a loopless reaction, parsimonious FBA will also have no flux through the loop.
## ref) https://cobrapy.readthedocs.io/en/latest/loopless.html#Loopless-model

start


In [ ]:
## Loopless FBA 분석_v2 이건 돌아간다? ##
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
add_loopless(iMsOB3b)
loop_sol = iMsOB3b.optimize()

## Loopless FBA flux값 저장 ###
model_flux(iMsOB3b, loop_sol)

In [ ]:
## Moma 설정 ## #####
## https://cobrapy.readthedocs.io/en/0.17.1/autoapi/cobra/flux_analysis/moma/index.html ##
#sol_moma = cobra.flux_analysis.moma(iMsOB3b)
#model_flux(iMsOB3b,sol_moma )

In [ ]:
## Essential_genes 찾기 ##

# Ojective Function 하고 싶은거 설정 #
iMsOB3b.objective = 'Biomass_Mextorquens_AM1_core'
# iMsOB3b.objective = 'PHAS'

##  Essential Gene 확인 ##
essen_gene_list = cobra.flux_analysis.find_essential_genes(iMsOB3b)
essen_gene_list

## Essential Reaction 확인 ##
essen_rxn_list = cobra.flux_analysis.find_essential_reactions(iMsOB3b)
len(essen_rxn_list)


852

In [ ]:
### single gene deletion 분석 ##
### Ref) https://cobrapy.readthedocs.io/en/0.17.1/autoapi/cobra/flux_analysis/moma/index.html
### 칼럼에 Growth라 써있지만 Objective의 Flux값을 의미하는 것으로 예상됨 - 아래 결과에서 PHAS 최대 Flux인 0.0340 값 확인 ###
iMsOB3b.objective = 'PHAS'
single_deletion = cobra.flux_analysis.deletion.single_reaction_deletion(iMsOB3b,
                                                      method = 'fba',
                                                      solution = 'moma',
                                                      )

,growth,status
ids,,
(MTHFD),2.430171e-17,optimal
(UAGCVT),0.000000e+00,optimal
(UAMAGS),0.000000e+00,optimal
(HACD7),3.400784e-02,optimal
(PAPPT3),0.000000e+00,optimal
...,...,...
(CH4trpp),3.400784e-02,optimal
(HACD1),3.400784e-02,optimal
(PPND),4.070938e-17,optimal


In [ ]:
### Robust Analysis?? ###
results_rb = cobra.flux_analysis.phenotype_phase_plane.production_envelope(iMsOB3b, 
                                                              reactions =['Biomass_Mextorquens_AM1_core' ,'PPND'], 
                                                              objective = 'PHAS',
                                                              carbon_sources = 'EX_ch4_e', 
                                                              points = 20)
results_rb

,carbon_source,flux_minimum,carbon_yield_minimum,mass_yield_minimum,flux_maximum,carbon_yield_maximum,mass_yield_maximum,Biomass_Mextorquens_AM1_core,PPND
0,EX_ch4_e,0.000000,0.0,NaN,0.000000,0.0,NaN,0.000000,0.000000
1,EX_ch4_e,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000489
2,EX_ch4_e,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000977
3,EX_ch4_e,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.001466
4,EX_ch4_e,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.001954
...,...,...,...,...,...,...,...,...,...
395,EX_ch4_e,NaN,NaN,NaN,NaN,NaN,NaN,0.122604,0.007328
396,EX_ch4_e,NaN,NaN,NaN,NaN,NaN,NaN,0.122604,0.007816
397,EX_ch4_e,NaN,NaN,NaN,NaN,NaN,NaN,0.122604,0.008305
398,EX_ch4_e,NaN,NaN,NaN,NaN,NaN,NaN,0.122604,0.008793


In [6]:
## optgene, optknock 돌리기 위한 Tool 설치##
!pip install cameo 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.9 MB 5.1 MB/s 
     |████████████████████████████████| 57 kB 4.2 MB/s 
     |████████████████████████████████| 88 kB 6.3 MB/s 
     |████████████████████████████████| 1.2 MB 55.0 MB/s 
     |████████████████████████████████| 76 kB 4.7 MB/s 
     |████████████████████████████████| 56 kB 3.7 MB/s 
     |████████████████████████████████| 231 kB 51.6 MB/s 
  Created wheel for gnomic: filename=gnomic-1.0.1-py3-none-any.whl size=14671 sha256=cfb1672cd8ea2272f53fc6c8055698e9d6d324f7d101d3b12ad7e77f77eef244
  Stored in directory: /root/.cache/pip/wheels/78/c5/31/10f917cc1ca63da8ca323fb2f79aa37d0cc75211561beb9d4c
Successfully built gnomic
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 4.3.3
    Uninstalling jsonschema-4.3.3:
      Successfully uninstalled jsonschema-4.3.3
  Attempting uninstall: pluggy
    Found existing in

In [ ]:
## Optgene ## A Methond using evolutionary algorithms
from cameo.strain_design import OptGene
optgene = OptGene(iMsOB3b)

result = optgene.run(target=iMsOB3b.reactions.PHAS,
                     biomass=iMsOB3b.reactions.Biomass_Mextorquens_AM1_core,
                     substrate=iMsOB3b.metabolites.ch4_e, ## EX_ch4_e ###
                     max_evaluations=5000,
                     plot=False)
result

Starting optimization at Thu, 09 Jun 2022 18:10:09


HBox()

Finished after 00:04:54


No solutions found


,reactions,genes,size,fva_min,fva_max,target_flux,biomass_flux,yield,fitness


In [ ]:
## OptKnock ## A method using linear programming ###
from cameo.strain_design import OptKnock
optknock = OptKnock(iMsOB3b, fraction_of_optimum=0.1)

result = optknock.run(max_knockouts=2,
                     target=iMsOB3b.reactions.PHAS,
                     biomass=iMsOB3b.reactions.Biomass_Mextorquens_AM1_core,
                    # substrate=iMsOB3b.metabolites.ch4_e, ## EX_ch4_e ###
                     #max_evaluations=5000,
                     #plot=False,
                      )
result

/usr/local/lib/python3.7/dist-packages/cameo/strain_design/deterministic/linear_programming.py:119 UserWarning: You are trying to run OptKnock with glpk_interface. This might not end well.


HBox()

,reactions,size,PHAS,biomass,fva_min,fva_max
0,"{NTPP7, CYOD}",2,0.034008,0.122604,0.033668,0.034008


In [ ]:
############# 기본 입력 방식 ####################
## 특정 반응식(PPND) 확인
# iMsOB3b.reactions.PPND.reaction

## 특정 반응식(PPND) 변경
# iMsOB3b.reactions.PPND.reaction = "ch4_p + o2_p + qh2_c --> h2o_p + meoh_p + q_c"
## 처음 넣는 대사체로 변경할 경우, unknown metabolite "xx" created 라는 말이 뜬다

## 특정 대사물질(q_c)이 포함된 반응식을 모두 뽑아내는 방법 ###
# for reaction in iMsOB3b.metabolites.q_c.reactions:
#     print(reaction.name, reaction)

## 반응식에 대사체 추가하기 ##
# iMsOB3b.reactions.PPND.add_metabolites({model.metabolites.get_by_id("h_c"): -1})
# iMsOB3b.reactions.PPND.reaction


## bound 확인##
# iMsOB3b.reactions.PGI.lower_bound,  iMsOB3b.reactions.PGI.upper_bound
## bound 변경##
# pgi.bounds = (0, 1000.0)

## deletion 처리할때
# AM1_bigg.reactions.PMMODCipp.knock_out()


## 특정 Reaction 뽑아낼 때, list comprehension 사용 ##
# qh2_c_consuming = [rxn for rxn in model_OB3b.metabolites.qh2_c.reactions if rxn.reversibility or model_OB3b.metabolites.qh2_c in rxn.reactants]

## 모델 Reaction에서 특정 단어가 들어간걸 찾을 떄 쓰는 방식.. ##
# import re
# for reaction in iMsOB3b.reactions:
#     if re.search("translocatin", reaction.name, re.IGNORECASE):
#         print(reaction.id, reaction.name)

NADH ubiquinone oxidoreductase I NADH11: 5.0 h_c + nadh_c + q_c --> 4.0 h_p + nad_c + qh2_c
succinate dehydrogenase SUCCD1: q_c + succ_c <=> fum_c + qh2_c
QRr QRr: h_c + nadph_c + q_c --> nadp_c + qh2_c
Nitrate reductase (Ubiquinol-8) NO3R1: no3_c + qh2_c --> h2o_c + no2_c + q_c
ETF dehydrogenase ETFDH: q_c + rETF_c <=> oxETF_c + qh2_c
particulate Methane Monooxygenase PMMOipp: ch4_p + o2_p + qh2_c --> h2o_p + meoh_p + q_c
particulate ammonia monooxygenase PAMMOipp: nh4_p + o2_p + qh2_c --> h2o_p + h_p + ham_p + q_c
ubiquinol cytochrome c reductase UQCYOR_2p: 2.0 ficytc_c + qh2_c --> 2.0 focytc_c + 2.0 h_p + q_c
cytochrome bd ubiquinol oxidase CYOD: 4.0 h_c + o2_c + 2.0 qh2_c --> 2.0 h2o_c + 4.0 h_p + 2.0 q_c
cytochrome o ubiquinol oxidase CYOO_4p: 4.0 h_c + 0.5 o2_c + qh2_c --> h2o_c + 4.0 h_p + q_c
photosystem I cyclic PHOTOSYS1C: 2.0 fdxrd_c + 2.0 h_c + q_c --> 2.0 fdxox_c + qh2_c
NADPH Quinone Reductase (Ubiquinone-8) NADPHQR2: h_c + nadph_c + q_c --> nadp_c + qh2_c


(-1000.0, 1000.0)

In [ ]:
## 모델에 새로운 Reaction 추가하기 ##
#####
# Step 1) 타? 모델에 있는 rxn을 가져오기...?
# iMsOB3b_adapted_reactions = [iMsOB3b.reactions.SAMMOi, iMsOB3b.reactions.NO3t7pp, iMsOB3b.reactions.NO3R1, 
#                              iMsOB3b.reactions.NITR_NOpp, iMsOB3b.reactions.NITR_AMpp, 
#                              iMsOB3b.reactions.NTRIR2y, iMsOB3b.reactions.NTRIR2x]

# Step 2) 쓸 모델에 add하기...  add_reaction은  Cobra.core.reaction인가? 이 포멧만 넣을 수 있음.. (출처 논문..)
# for reaction in iMsOB3b_adapted_reactions:
#     model_OB3b.add_reaction(reaction)
    
# Step 3) 그리고 reaction 식 바꾸기.. 아래처럼..
# model_OB3b.reactions.SAMMOi.reaction = "ch4_p + o2_p + qh2_c --> h2o_p + meoh_p + q_c"

In [ ]:
## 원랜 신규 Reaciton 만드는 방법은 아래와 같다.... 복잡... ##
from cobra import Model, Reaction, Metabolite

reaction = Reaction('R_3OAS140')
reaction.name = '3 oxoacyl acyl carrier protein synthase n C140 '
reaction.subsystem = 'Cell Envelope Biosynthesis'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

ACP_c = Metabolite(
    'ACP_c',
    formula='C11H21N2O7PRS',
    name='acyl-carrier-protein',
    compartment='c')
omrsACP_c = Metabolite(
    'M3omrsACP_c',
    formula='C25H45N2O9PRS',
    name='3-Oxotetradecanoyl-acyl-carrier-protein',
    compartment='c')
co2_c = Metabolite('co2_c', formula='CO2', name='CO2', compartment='c')
malACP_c = Metabolite(
    'malACP_c',
    formula='C14H22N2O10PRS',
    name='Malonyl-acyl-carrier-protein',
    compartment='c')
h_c = Metabolite('h_c', formula='H', name='H', compartment='c')
ddcaACP_c = Metabolite(
    'ddcaACP_c',
    formula='C23H43N2O8PRS',
    name='Dodecanoyl-ACP-n-C120ACP',
    compartment='c')



reaction.add_metabolites({
    malACP_c: -1.0,
    h_c: -1.0,
    ddcaACP_c: -1.0,
    co2_c: 1.0,
    ACP_c: 1.0,
    omrsACP_c: 1.0
})


reaction.gene_reaction_rule = '( STM2378 or STM1197 )'


### 간단 추가 버전 ##
from cobra import Model, Reaction, Metabolite
# Best practise: SBML compliant IDs

reaction = Reaction('PFK')
reaction.name = 'Phosphofructokinase'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default

iMsOB3b.add_reaction(reaction)
iMsOB3b.reactions.PFK.reaction = 'atp_c + f6p_c <=> adp_c + fdp_c + h_c'
iMsOB3b.reactions.FDHr.gene_reaction_rule = 'CQW49_16615 and CQW49_16630 and (CQW49_16625 or CQW49_16125) and CQW49_16635'

In [ ]:
### KEGG 적용 ###
import orangecontrib.bioinformatics.kegg

organism = orangecontrib.bioinformatics.kegg.Organism("Methylosinus trichosporium")
genes = organism.genes
gene_entry = genes["CQW49_13850"]
print(gene_entry.entry, gene_entry.definition)

organism = orangecontrib.bioinformatics.kegg.Organism("Methylosinus trichosporium")
genes = organism.genes
gene_entry = genes["CQW49_12210"]
print(gene_entry.entry, gene_entry.orthology)


organism = orangecontrib.bioinformatics.kegg.Organism("Methylosinus trichosporium")
genes = organism.genes
gene_entry = genes["CQW49_04750"]
print(gene_entry.entry, gene_entry.definition, gene_entry.orthology)


enzyme=orangecontrib.bioinformatics.kegg.databases.Enzyme()
enzyme_entry = enzyme["ec:2.3.3.1"]
print(enzyme_entry.reaction)

In [ ]:
#script for renaming record description in Refseq or Genbank files

#importing Biopythom SeqIO library
from Bio import SeqIO

#loading multiple gene elements from a fasta file
records = list(SeqIO.parse(fastas_directory + "OB3b/New_2017_assembly/GenBank_ASM275265v1_translated_cds.faa", 
                           "fasta"))


## 이렇게 가져와서, 기존 모델을 바꿔서 GEM 모델을 만들기도..

In [7]:
!pip install escher

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
!jupyter nbextension list

Known nbextensions:
  config dir: /usr/local/etc/jupyter/nbconfig
    notebook section
      escher/extension  enabled 
      - Validating: OK
      jupyterlab-plotly/extension  enabled 
      - Validating: OK
      jupyter-js-widgets/extension  enabled 
      - Validating: OK


In [17]:
import escher
from escher import Builder
from time import sleep
#from google.colab import output
#output.enable_custom_widget_manager()
builder = Builder(
    model_json = r'/content/gem_methylosinus_trichosporium/model_files/iMsOB3b.json',
    map_json = r'/content/gem_methylosinus_trichosporium/escher_metabolic_map/Methylosinus_trichosporium_OB3b_map.json')
builder

## https://escher.readthedocs.io/en/latest/escher-python.html

Builder()